# Julia Things

### Environment

First things first. Let us set up the environment with the requried packages for this notebook. We will also set the desired context (e.g. `KnetArray` for gpu), the number of epochs (`nepochs`), and the variable `fast`. This variable is used to skip checking the accuracy at every epoch. 

In [1088]:
for p in ("Knet", "Plots", "DataFrames")
    Pkg.installed(p) == nothing && Pkg.add(p)
end

using Knet, Plots, DataFrames, DSP
gr()

Knet.gpu(0); # set the desired GPU to use
atype   = Array{Float64}; # atype = KnetArray{Float32} for gpu usage, Array{Float32} for cpu. 
nepochs = 10
fast    = false

println("OS: ", Sys.KERNEL)
println("Julia: ", VERSION)
println("Knet: ", Pkg.installed("Knet"))
println("GPU: ", readstring(`nvidia-smi --query-gpu=name --format=csv,noheader`))

OS: Linux
Julia: 0.6.0
Knet: 0.8.5+
GPU: NVS 310
TITAN X (Pascal)



### New Stuff

In this notebook we introduce the following Julia/Knet packages and functions:

* Julia's [DSP](https://github.com/JuliaDSP/DSP.jl) package: Filter design, periodograms, window functions, and other digital signal processing functionality. 
* DSP's functions `Bandpass`, `Butterworth`, `digitalfilter`, and `filt`. 

# Exponential Smoothing and Innovation State Space Model (ISSM)

In this notebook we will illustrate the implementation of filtering in innovation state space model (ISSM, for short) using Knet. Let us first briefy reivew the basic concepts.


Time series forecasting is a central problem occuring in many applications from optimal inventory management, staff scheduling to topology planning. 
Given a sequence of measurements $z_1, \ldots, z_T$ observed over time, the problem here is to predict future values of the time series $z_{T+1}, \ldots, z_{T+\tau}$, where $\tau$ is referred as the *time horizon*.

Exponential smoothing (ETS, which stands for *Error, Trend, and Seasonality*) is a family of very successful forecasting methods which are based on the key property that forecasts are weighted combinations of past observations ([Hyndman et. al, 2008](http://www.exponentialsmoothing.net/home)).

For example, in simple exponential smoothing, the foreacast $\hat{z}_{T+1}$ for time step $T+1$ is written as ([Hyndman, Athanasopoulos, 2012](https://www.otexts.org/fpp/7/1))

$$ \hat{z}_{T+1} = \hat{z}_T + \alpha (z_T - \hat{z}_T) = \alpha\cdot z_T + (1 - \alpha)\cdot \hat{z}_T, $$

In words, the next step forecast is a convex combination of the most recent obseravtion and forecast. Expanding the above equation, it is clear that the forecast is given by the exponentially weighted average of past observations, 

$$ \hat{z}_{T+1} = \alpha z_T + \alpha(1-\alpha) z_{T-1} + \alpha(1-\alpha)^2 z_{T-2}+ \cdots. $$

Here $\alpha > 0$ is a smoothing parameter that controls the weight given to each observation.
Note that the recent observations are given more weight than the older observations.
In fact the weight given to the past observation descreases exponentially as it gets older and hence the name **exponential smoothing**.

General exponential smoothing methods consider the extensions of simple ETS to include time series patterns such as (linear) trend, various periodic seasonal effects. All ETS methods falls under the category of forecasting methods as the predictions are point forecasts (a single value is predicted for each future time step). 
On the other hand a statistical model describes the underlying data generation process and has an advantage that it can produce an entire probability distribuiton for each of the future time steps.
Innovation state space model (ISSM) is an example of such models with considerable flexibility in respresnting commonly occurring time series patterns and underlie the exponential smoothing methods.

The idea behind ISSMs is to maintain a latent state vector $l_{t}$ with recent information about level, trend, and seasonality factors.
The state vector $l_t$ evolves over time adding small *innvoation* (i.e., the Gaussian noise) at each time step. 
The observations are then a linear combination of the components of the current state.

Mathematically, ISSM is specified by two equations

* The state transition equation is given by 

$$l_{t} = F_t l_{t-1} + g_{t}\epsilon_t,\quad \epsilon_t\sim \mathcal{N}(0,1).$$

Note that the innovation strength is controlled by $g_t$, i.e., $g_t\epsilon_t \sim \mathcal{N}(0, g_t^2)$.

* The observation equation is given by

$$z_t = a_{t}^{\top}l_{t-1} + b_t + \nu_t, \quad \nu_t \sim \mathcal{N}(0, \sigma_t^2)$$

Note that here we allow for an additional term $b_t$ which can model any determinstic component (exogenous variables).

This describes a fairy generic model allowing the user to encode specific time series patterns using the coefficients $F$, $a_t$ and thus are problem dependent. The innovation vector $g_t$ comes in terms of parameters to be learned (the innovation strengths). Moreover, the initial state $l_0$ has to be specified. 
We do so by specifying a Gaussian prior distribution $P(l_0)$, whose parameters (means, standard deviation) are learned from data as well.

The parameters of the ISSM are typically learned using the maximum likelihood principle. 
This requires the computation of the log-likelihood of the given observations i.e., computing the probability of the data under the model, $P(z_1, \ldots, z_T)$. Fortunately, in the previous notebook, we have learned how to compute the log-likelihood as a byproduct of LDS filtering problem. 



# Filtering

We remark that ISSM is a special case of linear dynamical system except that the coefficients are allowed to change over time. The filtering equations for ISSM can readily be obtained from the general derivation described in LDS.


Note the change in the notation in the following equations for filtered mean ($\mu_t$) and filtered variance ($S_t$) because of the conflict of notation for the ISSM coefficient $F$. Also note that the deterministic part $b_t$ needs to be subtracted from the observations $[z_t]$. 

$$\mu_h = F_t \mu_{t-1} \quad \quad \quad \mu_v = a_t^{\top}\mu_h$$

$$\Sigma_{hh} = F_t S_{t-1}F_t^T + g_t g_t^T \quad \quad \quad \sigma^2_{v} = a_t^T\Sigma_{hh}a_t + \sigma_t^2$$

$$K_t = \frac{1} {\sigma^2_{v}} \Sigma_{hh}a_t $$

$$\mu_t = \mu_h + K(z_t - b_t -\mu_v) \quad \quad \quad S_t = (I - K_t a_t^T)\Sigma_{hh}(I-K_t a_t^T)^T +  \sigma^2_t K_tK_t^T$$






In [1089]:
function ISSM_filter(z, b, F, a, g, sigma, m_prior, S_prior)
    
    H = size(F, 1);
    T = size(z, 1)
    
    eye_h = eye(Float64, H)
    
    mu_seq    = []
    S_seq     = []
    log_p_seq = []
    mu_t      = 0
    S_t       = 0
    for t = 1:T
        
        if t == 1
            # At the first time step, use the prior
            mu_h = m_prior
            S_hh = S_prior
        else
            # Otherwise compute using update eqns.
            F_t = F[:, :, t];
            g_t = reshape(g[:, t], H, 1);
            mu_h = F_t * mu_t;
            S_hh = F_t * (S_t * F_t') + g_t * g_t';
        end
        
        
        a_t  = reshape(a[:, t], H, 1)
        mu_v = mu_h' * a_t;
        
        # Compute the Kalman gain (vector)
        S_hh_x_a_t = S_hh * a_t;

        sigma_t = sigma[t]
        S_vv    = a_t' * S_hh_x_a_t + sigma_t .^ 2

        kalman_gain = S_hh_x_a_t ./ S_vv

        # Compute the error (delta)
        delta = z[t] - b[t] - mu_v
        # Filtered estimates
        mu_t = mu_h .+ kalman_gain * delta;

        # Joseph's symmetrized update for covariance:
        
        ImKa = eye_h .- kalman_gain * a_t';


        S_t = (ImKa * S_hh) * ImKa' + (kalman_gain * kalman_gain') .* sigma_t .^ 2

        # likelihood term
        log_p = -0.5 * (delta * delta) ./ S_vv + log(2.0 + pi) + log.(S_vv)


        push!(mu_seq, mu_t)
        push!(S_seq, S_t)
        push!(log_p_seq, log_p)
    end
    
    return mu_seq, S_seq, log_p_seq

end

ISSM_filter (generic function with 1 method)

## Data

We will use the [beer shipment dataset](https://datamarket.com/data/set/2325/four-weekly-totals-of-beer-shipments#!ds=2325&display=line) to illustrate two specific instances of ISSM models.

In [1090]:
data = readtable("../datasets/fourweekly-totals-of-beer-shipme.csv");

In [1091]:
z  = Array{Float64}(data[:, 2])
ts = (z - mean(z)) / std(z);

In [1092]:
plot(data[:, 1], data[:, 2])

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1970-W06 
 
 
 1970-W22 
 
 
 1970-W34 
 
 
 1970-W50 
 
 
 1971-W09 
 
 
 1971-W25 
 
 
 1971-W37 
 
 
 1972-W01 
 
 
 1972-W17 
 
 
 1972-W29 
 
 
 1972-W45 
 
 
 1973-W05 
 
 
 1973-W21 
 
 
 1973-W33 
 
 
 1973-W49 
 
 
 20000 
 
 
 25000 
 
 
 30000 
 
 
 35000 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1

## Level ISSM

The simplest possible ISSM maintains a level component only. Abusing the notation and let $l_t$ denote *level*, the level ISSM can be written as

$$
\begin{split}
l_t = \delta l_{t-1} + \alpha \epsilon_t.
\end{split}
$$

Or in ISSM terminology, 
$$
  a_{t} = [ \delta ],\quad F_{t} = [ \delta ],\quad g_{t} = [ \alpha ],\quad \alpha>0.
$$

The level $l_t \in \mathbb{R}$ evolves over time by adding a random innovation $\alpha \epsilon_t \sim \mathcal{N}(0,\alpha^2)$ to the previous level, so that $\alpha$ specifies the amount of level drift over time. At time $t$, the previous level $l_{t-1}$ is used in the prediction $z_t$ and then the level is updated. 
The damping factor $\delta \in (0, 1]$ allows the ``damping'' of the level.
The initial state prior $P(l_0)$ is given by $l_0 \sim N(\mu_0, \sigma_0^2)$. For Level-ISSM, we learn the parameters $\alpha>0$, $\mu_0$, $\sigma_0>0$.



Here we will fix the parameters for the illustration of filtering.
Learning of the parameters will be discussed in another notebook.

In [1093]:
function initmodel(ts)

    latent_dim = 1;
    T          = length(ts);

    # Set the coefficients of the ISSM
    delta      = Float64(1.0)
    F          = delta * ones(Float64, 1, 1, T);
    a          = delta * ones(Float64, 1, T);

    # Set the parameters of the ISSM
    alpha      = Float64(0.5);
    g          = alpha * ones(Float64, 1, T);

    m_prior    = zeros(Float64, latent_dim, 1)
    S_prior    = zeros(Float64, latent_dim, latent_dim)
    sigma      = 0.5 * ones(Float64, T, 1)
    b          = zeros(Float64, T, 1)
    z          = copy(ts);

    return z, b, F, a, g, sigma, m_prior, S_prior
end

initmodel (generic function with 1 method)

In [1094]:
function ISSM_filter(z, b, F, a, g, sigma, m_prior, S_prior)
    
    H = size(F, 1);
    T = size(z, 1)
    
    eye_h = eye(Float64, H)
    
    mu_seq    = []
    S_seq     = []
    log_p_seq = []
    mu_t      = 0
    S_t       = 0
    for t = 1:T
        
        if t == 1
            # At the first time step, use the prior
            mu_h = m_prior
            S_hh = S_prior
        else
            # Otherwise compute using update eqns.
            F_t = F[:, :, t];
            g_t = reshape(g[:, t], H, 1);
            mu_h = F_t * mu_t;
            S_hh = F_t * (S_t * F_t') + g_t * g_t';
        end
        
        
        a_t  = reshape(a[:, t], H, 1)
        mu_v = mu_h' * a_t;
        
        # Compute the Kalman gain (vector)
        S_hh_x_a_t = S_hh * a_t;

        sigma_t = sigma[t]
        S_vv    = a_t' * S_hh_x_a_t + sigma_t .^ 2

        kalman_gain = S_hh_x_a_t ./ S_vv

        # Compute the error (delta)
        delta = z[t] - b[t] - mu_v
        # Filtered estimates
        mu_t = mu_h + kalman_gain * delta;

        # Joseph's symmetrized update for covariance:
        
        ImKa = eye_h .- kalman_gain * a_t';


        S_t = (ImKa * S_hh) * ImKa' + (kalman_gain * kalman_gain') .* sigma_t .^ 2

        # likelihood term
        log_p = -0.5 * (delta * delta) ./ S_vv + log(2.0 + pi) + log.(S_vv)


        push!(mu_seq, mu_t)
        push!(S_seq, S_t)
        push!(log_p_seq, log_p)
    end
    
    return mu_seq, S_seq, log_p_seq

end

ISSM_filter (generic function with 1 method)

In [1095]:
z, b, F, a, g, sigma, m_prior, S_prior = initmodel(ts)
mu_seq, S_seq, _ = ISSM_filter(z, b, F, a, g, sigma, m_prior, S_prior);

### Calculate the filtered mean and variance of observations


Given $p(l_{t-1}|z_{1:t})=\mathcal{N}(\mu_t, S_t)$, we can compute the distribution of the reconstructed observations 

$$
p(\widehat{z_t}) = \mathcal{N}(a_t^T\mu_t, a_t^TS_ta_t + \sigma_t^2).
$$

In [1096]:
function reconstruct(a, H, mu_seq, S_seq)
    
    v_filtered_mean = [(reshape(a[:,t], 1, H)*mu_t)[1] for (t,mu_t) in zip(1:length(mu_seq),mu_seq)];
    v_filtered_std  = [(reshape(a[:,t], 1, H)S_t*a[:,t]+ sigma[t].^2)[1] for (t,S_t) in zip(1:length(S_seq),S_seq)];
    v_filtered_std  = sqrt.(v_filtered_std);

    return v_filtered_mean, v_filtered_std

end

reconstruct (generic function with 3 methods)

In [1097]:
v_filtered_mean, v_filtered_std = reconstruct(a, size(F, 1), mu_seq, S_seq);

### Forecast

One advantage of the ISSM model is that one can obtain the complete probability distribution for each of the future time steps:

$$
p(\widehat{z_{T+t}}) = \mathcal{N}(a_{T+t}^T\mu_{T+t}, a_{T+t}^TS_{T+t}a_{T+t} + \sigma_{T+t}^2),\quad t > 0 \\
p(l_{T+t}) = \mathcal{N}(F\mu_{T+t-1}, FS_{T+t-1}F^T + g_{T+t} g_{T+t}^T)
$$

In [1098]:
function forecast(mu_last_state, S_last_state, F, a, g, sigma; horizon=13)

    forecasts_mean = []
    forecasts_std  = []
    H = size(F, 1);
    for t = 1:horizon
        a_t  = reshape(a[:, t], 1, H)
        forecast_mean = a_t * mu_last_state
        
        forecast_std  = a_t * S_last_state * a_t' + sigma[t] .^ 2
        push!(forecasts_mean, forecast_mean[1])
        push!(forecasts_std, forecast_std[1])
        mu_last_state = F[:, :, t] * mu_last_state
        S_last_state  = F[:, :, t] * S_last_state * F[:, :, t]'
    end
    
    return forecasts_mean, forecasts_std
    
end

forecast (generic function with 2 methods)

In [1099]:
# Let us use the same cofficients (constant over time) for the future as well
forecasts_mean, forecasts_std = forecast(mu_seq[end], 
                                         S_seq[end], 
                                          F, a, g, sigma; horizon=13);

In [1100]:
function plot_reconstruction_forecasts(ts, v_filtered_mean, v_filtered_std, forecasts_mean, forecasts_std)

    T = length(v_filtered_mean);
    plot(1:T, v_filtered_mean, grid=false, ribbon=v_filtered_std,fillalpha=.5, color=:blue, label=:_, legend=:bottomleft)
    plot!(ts, color=:red, label=:data)
    plot!(v_filtered_mean, color=:blue, label=:reconstruction)
    plot!(T:T+length(forecasts_mean) - 1, forecasts_mean, grid=false, ribbon=forecasts_std,fillalpha=.5, color=:green, label=:forecasts)

end

plot_reconstruction_forecasts (generic function with 1 method)

In [1101]:
plot_reconstruction_forecasts(ts, v_filtered_mean, v_filtered_std, forecasts_mean, forecasts_std)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 50 
 
 
 60 
 
 
 -2 
 
 
 -1 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 _ 
 
 
 
 data 
 
 
 
 reconstruction 
 
 
 
 forecasts

### Level Trend ISSM

We can model a piecewise linear random process by using a two-dimensional latent state $l_{t}\in \mathbb{R}^2$, where one dimension represents the level (again with a slight abusing of notation, $l$) and the other represents the trend (slope) $b$. 

$$
\begin{split}
  l_t &= \delta l_{t - 1} + \gamma b_{t - 1} + \alpha\cdot\epsilon_t\\
  b_t &= \gamma b_{t - 1} + \beta\cdot\epsilon_t
  \end{split}
$$


In ISSM framework, such a (Damped) LevelTrend-ISSM is given by

$$
  a_{t} = \left[\begin{array}{c}
    \delta \\
    \gamma
  \end{array}\right], \quad F_{t} = \left[\begin{array}{cc}
    \delta & \gamma \\
    0 & \gamma
  \end{array}\right], \quad g_{t} = \left[\begin{array}{c}
    \alpha \\
    \beta
  \end{array}\right],
$$

where $\alpha>0$, $\beta>0$ and the damping factors $\delta, \gamma \in (0, 1]$.
Both the level and slope components evolve over time by adding innovations $\alpha \epsilon_t$ and $\beta \epsilon_t$ respectively, so that $\beta>0$ is the innovation strength for the slope. The level at time $t$ is the sum of level at $t-1$ and slope at $t-1$ (linear prediction) modulo the damping factors for level $\delta$ and growth $\gamma$.

In [1169]:
function initmodel2(ts)
    latent_dim = 2
    T          = length(ts)

    # Set the coefficients of the ISSM
    damp_fact   = 1.0
    damp_growth = 1.0

    # Set the parameters of the ISSM
    alpha      = 0.5 
    beta       = 0.1 
    g_t        = [alpha, beta]
    g = reshape(repeat(g_t; inner=T), latent_dim, T);


    F_t = reshape([damp_fact, damp_growth, 0, damp_growth], latent_dim, latent_dim);
    a_t = [damp_fact, damp_growth];

    #F = reshape(repeat(F_t, outer=(T, 1), inner=(1,1)), latent_dim, latent_dim, T);
    F = Array{Float64}(2, 2, T)
    F[1, 1, :] = damp_fact*ones(1, T)
    F[1, 2, :] = damp_growth*ones(1, T)
    F[2, 1, :] = zeros(1, T)
    F[2, 2, :] = damp_growth*ones(1, T);

    a = reshape(repeat(a_t, inner=T), latent_dim, T);


    m_prior    = zeros(latent_dim, 1)
    S_prior    = zeros(latent_dim, latent_dim)
    sigma      = 0.5 * ones(T, 1)
    b          = zeros(T, 1)
    z          = copy(ts);
    
    return z, b, F, a, g, sigma, m_prior, S_prior
    
end

initmodel2 (generic function with 1 method)

In [ ]:
mu_seq, S_seq, _ = ISSM_filter(z, b, F, a, g, sigma, m_prior, S_prior);

In [1105]:
# Let us use the same cofficients (constant over time) for the future as well
forecasts_mean, forecasts_std = forecast(mu_seq[end], 
                                         S_seq[end], 
                                          F, a, g, sigma; horizon=13);

In [1106]:
reconst_mean, reconst_std = reconstruct(a, mu_seq, S_seq);

## Plot the reconstruction as well as the forecasts

In [1107]:
plot_reconstruction_forecasts(ts, v_filtered_mean, v_filtered_std, forecasts_mean, forecasts_std)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 50 
 
 
 60 
 
 
 -6 
 
 
 -4 
 
 
 -2 
 
 
 0 
 
 
 2 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 _ 
 
 
 
 data 
 
 
 
 reconstruction 
 
 
 
 forecasts

# Cardiac ECG Data

Let us try a different dataset. In this example we will use electrocardiogram (ECG) data from the [physiobank dataset](https://www.physionet.org/physiobank/annotations.shtml):

In [1124]:
data         = readtable("../datasets/ecg_data/MIT-BIH-arrhythmia-database/100.csv");
time_stamps  = readtable("../datasets/ecg_data/MIT-BIH-arrhythmia-database/100_ann.csv");
label_stamps = readtable("../datasets/ecg_data/MIT-BIH-arrhythmia-database/100_anntype.csv");

Let us extract long and short samples from this dataset:

In [1239]:
S = 256 * 1;
L = 256 * 10;

ECG_short = (data[1:S, 2] - mean(data[1:S, 2])) / std(data[1:S, 2]);
ECG_long  = (data[1:L, 2] - mean(data[1:L, 2])) / std(data[1:L, 2]);

It should be no surprise that noise can be a problem in ECG analysis. Fortunately, the signal-to-noise ratio is usually quite good in a person at rest. In an active person, however, there can be substantial low frequency (< 15 Hz) noise due to electrode motion, and high frequency (> 15 Hz) noise due to skeletal muscle activity. In addition, there is the possibility of noise at 60 Hz (or 50 Hz in some countries) and its harmonics due to power-line noise. We will filter the ECG data, sampled at 350 Hz, with a 4th order Butterworth bandpass filter between 5 and 30 Hz:

In [1240]:
responsetype = Bandpass(5, 30; fs=350)
designmethod = Butterworth(4)
ts_long = filt(digitalfilter(responsetype, designmethod), ECG_long);
ts      = filt(digitalfilter(responsetype, designmethod), ECG_short);

Let us visualize the data:

In [1241]:
l   = @layout([a{0.2h}; b])
hms = [[ts], [ts]]
plot([ts_long, ts], layout=l, legend=false, xlabel=:time, ylabel=:voltage)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 500 
 
 
 1000 
 
 
 1500 
 
 
 2000 
 
 
 2500 
 
 
 -4 
 
 
 -2 
 
 
 0 
 
 
 2 
 
 
 time 
 
 
 voltage 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip8302)" style="stroke:#009af9; stroke-width:1; str

In [1242]:
hms = []
for k in [2, 5, 6, 7]
    S = 256 * k;
    ECG_short = (data[1 + shift:S, 2] - mean(data[1 + shift:S, 2])) / std(data[1 + shift:S, 2]);

    responsetype = Bandpass(5, 30; fs=350)
    designmethod = Butterworth(4)
    ts      = filt(digitalfilter(responsetype, designmethod), ECG_short);

    z, b, F, a, g, sigma, m_prior, S_prior = initmodel2(ts)
    mu_seq, S_seq, _ = ISSM_filter(z, b, F, a, g, sigma, m_prior, S_prior);
    v_filtered_mean, v_filtered_std = reconstruct(a, size(F, 1), mu_seq, S_seq);
    forecasts_mean, forecasts_std = forecast(mu_seq[end], S_seq[end], F, a, g, sigma; horizon=20);

    N = 300
    push!(hms, plot_reconstruction_forecasts(ts[end-N: end], 
              v_filtered_mean[end-N: end], v_filtered_std[end-N: end],
              forecasts_mean, forecasts_std))
end

In [1243]:
l   = @layout([a b; c d])
#hms = [[ts], [ts]]
plot(hms..., layout=l, legend=false)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 50 
 
 
 100 
 
 
 150 
 
 
 200 
 
 
 250 
 
 
 300 
 
 
 -6 
 
 
 -4 
 
 
 -2 
 
 
 0 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 50 
 
 
 100 
 
 
 150 
 
 
 200 
 
 
 250 
 
 
 300 
 
 
 -6 
 
 
 -3 
 
 
<text style="fill:#000000; fill-opacity:1; font-fa